In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
device = 'cuda' if torch.cuda.is_available() else 'cpu'


compute_dtype = getattr(torch,'float16')
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16,
)

model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
original_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True,
    token=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    add_bos_token = True,
    use_fast = False
)
tokenizer.pad_token = tokenizer.eos_token

/home/default/workspace/new_finetune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [02:21<00:00, 47.11s/it]


In [7]:
from peft import PeftModel
adapter_path = "fine_tuned_mistral/checkpoint-300"
ft_model = PeftModel.from_pretrained(original_model,adapter_path,is_trainable=False)

In [4]:
context = "You will be provided a summary of a task performed by a behavior tree, and your objective is to express this behavior tree in XML format.\n"
task = """The behavior tree represents a robot's navigation system with arm activity. The robot must visit the location "ArucoStand", then follow the arucos given their IDs: 10, 1, and 7. Finally the robot reset the manipulator to the parking position and goes to the location "Parking". The only available actions that must be used in the behavior tree are: "MoveTo", "FollowAruco" and "ResetManipulator"."""

# One-shot example
example_task = """The behavior tree represents a robot's navigation system with arm activity. The robot must visit the location "Station A", then follow the aruco with ID=7. The only available actions that must be used in the behavior tree are: "MoveTo", "FollowAruco"."""
example_output = """
<root main_tree_to_execute = "MainTree" >
    <BehaviorTree ID="MainTree">
        <Sequence>
            <MoveTo location="Station A"/>
            <FollowAruco id="7"/>
        </Sequence>
    </BehaviorTree>
</root>
"""

In [5]:
## zero shot
zero_shot_prompt = f"<s>[INST]{context} summary:{task}[/INST]"
zero_shot_input = tokenizer(zero_shot_prompt,return_tensors='pt').to(device)

##one shot
one_shot_prompt = f"<s>[INST]{context} example task:{example_task} \nexample_output:{example_output} \nsummary:{task}[/INST]"
one_shot_input = tokenizer(one_shot_prompt,return_tensors='pt').to(device)

In [8]:
import re
 
# Define regular expression pattern to extract the behavior tree from the complete output
def print_bt(output):
    pattern = r'<root .*?</root>'
    matches = re.findall(pattern, output, re.DOTALL)
    
    if matches:
        final_tree = matches[-1]
        print(final_tree)

## Base model inference

In [6]:
original_model.eval()
with torch.no_grad():
    result = tokenizer.decode(original_model.generate(**zero_shot_input,max_new_tokens=1000)[0],skip_special_tokens=True)
    print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]You will be provided a summary of a task performed by a behavior tree, and your objective is to express this behavior tree in XML format.
 summary:The behavior tree represents a robot's navigation system with arm activity. The robot must visit the location "ArucoStand", then follow the arucos given their IDs: 10, 1, and 7. Finally the robot reset the manipulator to the parking position and goes to the location "Parking". The only available actions that must be used in the behavior tree are: "MoveTo", "FollowAruco" and "ResetManipulator".[/INST]

output:<root main_tree_to_execute = "MainTree">
    <BehaviorTree ID="MainTree">
        <Sequence>
            <Action ID="MoveTo" goal="ArucoStand"/>
            <Action ID="FollowAruco" goal_id="10"/>
            <Action ID="FollowAruco" goal_id="1"/>
            <Action ID="FollowAruco" goal_id="7"/>
            <Action ID="ResetManipulator"/>
            <Action ID="MoveTo" goal="Parking"/>
        </Sequence>
    </BehaviorTree>
</r

In [9]:
print_bt(result)

<root main_tree_to_execute = "MainTree">
    <BehaviorTree ID="MainTree">
        <Sequence>
            <Action ID="MoveTo" goal="ArucoStand"/>
            <Action ID="FollowAruco" goal_id="10"/>
            <Action ID="FollowAruco" goal_id="1"/>
            <Action ID="FollowAruco" goal_id="7"/>
            <Action ID="ResetManipulator"/>
            <Action ID="MoveTo" goal="Parking"/>
        </Sequence>
    </BehaviorTree>
</root>


## Fine-tuned model

In [10]:
ft_model.eval()
with torch.no_grad():
    result = tokenizer.decode(ft_model.generate(**zero_shot_input,max_new_tokens=1000)[0],skip_special_tokens=True)
    print_bt(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<root main_tree_to_execute = "MainTree">
    <BehaviorTree ID="MainTree">
        <Sequence>
            <Action ID="MoveTo" goal="ArucoStand"/>
            <Action ID="FollowAruco" goal_id="10"/>
            <Action ID="FollowAruco" goal_id="1"/>
            <Action ID="FollowAruco" goal_id="7"/>
            <Action ID="ResetManipulator"/>
            <Action ID="MoveTo" goal="Parking"/>
        </Sequence>
    </BehaviorTree>
</root>


## Random GPT Example

In [11]:
context = "You will be provided a summary of a task performed by a behavior tree, and your objective is to express this behavior tree in XML format.\n"
task = "Imagine a domestic cleaning robot named RoboCleaner. RoboCleaner is designed to efficiently clean various rooms in a house. It has sensors to detect obstacles, dirt, and walls. The robot's task is to navigate through the rooms, avoid obstacles, and clean the surfaces effectively."
zero_shot_prompt = f"<s>[INST]{context} summary:{task}[/INST]"
zero_shot_input = tokenizer(zero_shot_prompt,return_tensors='pt').to(device)

original_model.eval()
with torch.no_grad():
    result = tokenizer.decode(original_model.generate(**zero_shot_input,max_new_tokens=1000)[0],skip_special_tokens=True)
    print_bt(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<root main_tree_to_execute = "MainTree" >
    <BehaviorTree ID="MainTree">
        <Sequence name="root_sequence">
            <!-- Room 1 -->
            <Fallback name="room1_fallback">
                <Inverter name="is_room1_clean">
                    <Sequence name="room1_sequence">
                        <OpenDoor name="open_door_to_room1"/>
                        <Move name="move_to_room1"/>
                        <ActivateCleaning name="activate_cleaning"/>
                    </Sequence>
                </Inverter>
            </Fallback>
            <!-- Room 2 -->
            <Fallback name="room2_fallback">
                <Inverter name="is_room2_clean">
                    <Sequence name="room2_sequence">
                        <OpenDoor name="open_door_to_room2"/>
                        <Move name="move_to_room2"/>
                        <ActivateCleaning name="activate_cleaning"/>
                    </Sequence>
                </Inverter>
            </Fallback>

In [12]:
ft_model.eval()
with torch.no_grad():
    result = tokenizer.decode(ft_model.generate(**zero_shot_input,max_new_tokens=1000)[0],skip_special_tokens=True)
    print_bt(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<root main_tree_to_execute = "MainTree" >
    <BehaviorTree ID="MainTree">
        <Sequence name="root_sequence">
            <!-- Room 1 -->
            <Fallback name="room1_fallback">
                <Inverter name="is_room1_clean">
                    <Sequence name="room1_sequence">
                        <OpenDoor name="open_door_to_room1"/>
                        <Move name="move_to_room1"/>
                        <ActivateCleaning name="activate_cleaning"/>
                    </Sequence>
                </Inverter>
            </Fallback>
            <!-- Room 2 -->
            <Fallback name="room2_fallback">
                <Inverter name="is_room2_clean">
                    <Sequence name="room2_sequence">
                        <OpenDoor name="open_door_to_room2"/>
                        <Move name="move_to_room2"/>
                        <ActivateCleaning name="activate_cleaning"/>
                    </Sequence>
                </Inverter>
            </Fallback>